## Going to focus on OPRK1

In [1]:
# Library time 
import pandas as pd
import numpy as np
import atomsci.ddm.pipeline.model_pipeline as mp
import atomsci.ddm.pipeline.parameter_parser as parse
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.utils.struct_utils as struct_utils
from atomsci.ddm.pipeline import perf_plots as pp

2021-11-05 00:18:16.156662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/spack/brown/apps/proj/5.2.0-gcc-4.8.5-usib7od/lib:/apps/spack/brown/apps/geos/3.7.2-gcc-4.8.5-3vanyva/lib:/apps/spack/brown/apps/gdal/2.4.2-gcc-4.8.5-uj736h3/lib:/apps/spack/brown/apps/netcdf/4.5.0-gcc-6.3.0-d6fczmr/lib:/apps/spack/brown/apps/libtiff/4.0.10-gcc-6.3.0-6p5trqs/lib:/apps/spack/brown/apps/hdf5/1.8.16-gcc-6.3.0-7q7ndrz/lib:/apps/spack/brown/apps/hdf/4.2.14-gcc-6.3.0-2xg7pyg/lib:/scratch/brown/kamstut/tdm/apps/jupyter/kernels/llnl/.venv/lib:/apps/spack/brown/apps/r/4.0.0-gcc-6.3.0-hrvmcqp/rlib/R/lib:/apps/spack/brown/apps/openblas/0.3.7-gcc-6.3.0-qk24sho/lib:/apps/spack/brown/apps/tk/8.6.8-gcc-6.3.0-6qaesqb/lib:/apps/spack/brown/apps/tcl/8.6.8-gcc-6.3.0-n6mxabo/lib:/apps/spack/brown/apps/zlib/1.2.11-gcc-4.8.5-pkmj6e7/lib:/app

### First, let me split our dataset

In [25]:
ddir='/home/rwilfong/' # input directory 
file3=ddir+'OPRK1.csv' #file is the input directory plus the csv
# add a "compound_id" column that is a numerical counter and rename the old compound_id column as SMILES then save new file.
df=pd.read_csv(file3)
df.rename(columns={'compound_id' : 'SMILES'},inplace=True)
df['compound_id']=df.index
ofile=ddir+'mod_OPRK1.csv'
df.to_csv(ofile,index=False)
train_file=ofile
# now run splitter on modified file.
response_col = "target_OPRK1_standard_value"
compound_id = "compound_id"
smiles_col = "SMILES"
params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "previously_split": "False",
        "split_only": "True",
        "featurizer": "ecfp",
        "verbose": "True",
        "response_cols" : response_col,
        "transformers": "True",
        "rerun": "False",
        "result_dir": "split",
        "split_valid_frac": "0.10",           #fraction reserved for the validation set
        "split_test_frac": "0.10"             #fraction reserved for the test set
    }


In [26]:
ampl_param = parse.wrapper(params)

In [27]:
pl = mp.ModelPipeline(ampl_param)

In [28]:
save=pl.split_dataset()

2021-11-05 00:32:43,193 Attempting to load featurized dataset
2021-11-05 00:32:43,244 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-11-05 00:32:43,247 Featurized dataset not previously saved for dataset mod_OPRK1, creating new
2021-11-05 00:32:43,337 Featurizing sample 0
2021-11-05 00:32:46,389 Featurizing sample 1000
2021-11-05 00:32:49,411 Featurizing sample 2000
2021-11-05 00:32:52,428 Featurizing sample 3000
2021-11-05 00:32:55,465 Featurizing sample 4000
2021-11-05 00:32:58,499 Featurizing sample 5000
2021-11-05 00:33:01,512 Featurizing sample 6000
2021-11-05 00:33:04,657 Featurizing sample 7000
2021-11-05 00:33:07,734 Featurizing sample 8000
2021-11-05 00:33:10,589 Splitting data by scaffold


number of features: 1024


2021-11-05 00:33:16,605 Dataset split table saved to /home/rwilfong/mod_OPRK1_train_valid_test_scaffold_f5ac1b84-c990-43a0-8046-19173f0de54c.csv


In [29]:
print(save) 

f5ac1b84-c990-43a0-8046-19173f0de54c


In [30]:
split_uuid = 'f5ac1b84-c990-43a0-8046-19173f0de54c'

### Okay cool, now we have our split dataset for OPRK1!

My CSV file is `mod_OPRK1_train_valid_test_scaffold_f5ac1b84-c990-43a0-8046-19173f0de54c`

And my split ID is **f5ac1b84-c990-43a0-8046-19173f0de54c**

The scaffold portion in front just indicates the type of split performed. In this case, we used a scaffold split

### Let's make a graph convolutional neural network for OPRK1

In [65]:
split_uuid = '/home/rwilfong/mod_OPRK1_train_valid_test_scaffold_f5ac1b84-c990-43a0-8046-19173f0de54c.csv'
response_col = "target_OPRK1_standard_value"
train_file = '/home/rwilfong/mod_OPRK1.csv'
compound_id = "compound_id"
smiles_col = "SMILES"


train_config = {
    "script_dir": "/home/rwilfong/",
    "dataset_key" : train_file,
    "datastore": "False",
    "uncertainty": "False",
    "splitter": "scaffold",
    "split_valid_frac": "0.10",
    "split_test_frac": "0.10",
    "previously_split": "True",
    "split_uuid": split_uuid,
    "prediction_type": "regression",
    "response_cols" : response_col,
    "id_col": compound_id,
    "smiles_col" : smiles_col,
    "result_dir": "/home/rwilfong/ATOM/opioid/",
    "system": "LC",
    "transformers": "True",
    "model_type": "NN",
    "featurizer": "graphconv",
    "descriptor_type": "graphconv",
    "learning_rate": "0.0007",
    "layer_sizes": "64,64,32",
    "dropouts" : "0.0,0.0,0.0",
    "save_results": "False",
    "max_epochs": "100",
    "verbose": "True"
}

In [66]:
train_params = parse.wrapper(train_config)

In [67]:
train_model = mp.ModelPipeline(train_params)

In [68]:
train_model.train_model()

2021-11-05 01:20:24,089 Attempting to load featurized dataset
2021-11-05 01:20:25,098 Exception when trying to load featurized data:
DynamicFeaturization doesn't support get_featurized_dset_name()
2021-11-05 01:20:25,103 Featurized dataset not previously saved for dataset mod_OPRK1, creating new
2021-11-05 01:20:31,707 Featurizing sample 0
2021-11-05 01:20:35,452 Featurizing sample 1000
2021-11-05 01:20:38,378 Featurizing sample 2000
2021-11-05 01:20:41,302 Featurizing sample 3000
2021-11-05 01:20:44,258 Featurizing sample 4000
2021-11-05 01:20:47,187 Featurizing sample 5000
2021-11-05 01:20:52,679 Featurizing sample 6000
2021-11-05 01:20:56,029 Featurizing sample 7000
2021-11-05 01:20:59,126 Featurizing sample 8000


number of features: 75


2021-11-05 01:21:03,935 Error when loading dataset split table:
[Errno 2] No such file or directory: '/home/rwilfong/mod_OPRK1_train_valid_test_scaffold_/home/rwilfong/mod_OPRK1_train_valid_test_scaffold_f5ac1b84-c990-43a0-8046-19173f0de54c.csv.csv'
2021-11-05 01:21:03,938 Splitting data by scaffold
2021-11-05 01:21:18,999 Dataset split table saved to /home/rwilfong/mod_OPRK1_train_valid_test_scaffold_90562095-97fd-49cd-9b6b-797bfe313af0.csv
2021-11-05 01:21:19,090 Wrote transformers to /home/rwilfong/ATOM/opioid/mod_OPRK1/NN_graphconv_scaffold_regression/d0a06489-d683-4492-863e-2aa5e0a808ca/transformers.pkl
2021-11-05 01:21:19,091 Transforming response data
2021-11-05 01:21:21,304 Transforming response data
2021-11-05 01:21:21,607 Transforming response data
/scratch/brown/kamstut/tdm/apps/jupyter/kernels/llnl/.venv/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/priv

In [69]:
pred_data = train_model.model_wrapper.get_perf_data(subset="valid", epoch_label="best")
pred_results = pred_data.get_prediction_results()
print(f"R2 score of validation set: {pred_results['r2_score']:.3f}")

R2 score of validation set: 0.468


mod_OPRK1_train_valid_test_scaffold_90562095-97fd-49cd-9b6b-797bfe313af0